In [513]:
import jax
import jax.numpy as np
from jax import vmap,grad
from functools import partial
from jax.example_libraries import stax
from jax.example_libraries.stax import Dense,Relu,Flatten
#from jax import random
import random
from jax import jit
from keras.datasets import mnist,fashion_mnist
import numpy as onp
from jax.example_libraries import optimizers
from jax.tree_util import tree_multimap

In [384]:
features=data1.shape[1] #number of causal snps
reg_weight=1
in_shape=(-1,features)
ethnic_grp_min_pop=66 #min population among all subpopulatiosn
batch_size=50 #inner batch size for inner loop
K=20 #K-shot learning
num_task_sample= 4 #number of tasks to sample to meta train
lr=0.1
rng=jax.random.PRNGKey(1)

In [ ]:
#net_init,net_apply=stax.serial(
#    Flatten,Dense(2),Relu,Dense(num_classes),stax.elementwise(jax.nn.log_softmax))

In [369]:
net_init,net_apply=stax.serial(
  Dense(1) )

In [370]:
out_shape, net_params=net_init(rng,input_shape=in_shape)

In [ ]:
#for non-linear model 
#def cross_entropy(logprobs, targets):
#  nll = np.take_along_axis(logprobs, np.expand_dims(targets, axis=1), axis=1)
#  ce = -np.mean(nll)
#  return ce

#def loss(params,batch):
 #   inputs, targets = batch
  #  predictions=net_apply(params,inputs)
   # return cross_entropy(predictions,targets)

In [ ]:
#training_data,test_data=mnist.load_data()

In [371]:
def loss(params,data,targets):
    predictions=net_apply(params,inputs)
    for i in range(len(net_params)):
        l1_params=np.linalg.norm(net_params[i],1)
    return np.mean((targets-predictions)**2) + reg_weight*np.linalg.norm(net_params[0][0],1)

In [372]:
def inner_update(p,x1,y1):
    grads= grad(loss)(p,x1,y1)
    inner_sgd_fn= lambda g, state: (state-lr*g)
    return tree_multimap(inner_sgd_fn,grads,p)

def maml_loss(p,x1,y1,x2,y2):
    p2= inner_update(p,x1,y1)
    return loss(p2,x2,y2)

In [373]:
opt_init, opt_update, get_params = optimizers.adam(step_size=1e-3)
out_shape, net_params= net_init(rng,in_shape)
opt_state = opt_init(net_params)

@jit
def step(i,opt_state,x1,y1,x2,y2):
    p=get_params(opt_state)
    g= grad(maml_loss)(p,x1,y1,x2,y2)
    l=maml_loss(p,x1,y1,x2,y2)
    return opt_update(i,g,opt_state),l

In [374]:
def batch_maml_loss(p, x1_b, y1_b, x2_b, y2_b):
    task_losses = vmap(partial(maml_loss, p))(x1_b, y1_b, x2_b, y2_b)
    return np.mean(task_losses)

In [507]:
#need ethnic_grp_pop list which contains population of each ethinc group
#training data is a list of arrays each of which corresponds to an ethinic groups 
#inner batch size < ethnic_grp_min_pop
def sample_tasks(outer_batch_size, inner_batch_size):
    # Select amplitude and phase for the task
    ethnic_grp_sample=random.sample(eth, size=outer_batch_size)

    def get_batch():
        xs, ys = [], []
        for j in ethnic_grp_sample:
            indices = onp.random.randint(ethnic_grp_min_pop,size=inner_batch_size)
            x= x_train.loc[j].iloc(indices).to_numpy()
            y= y_train.loc[j].iloc(indices).to_numpy()
            xs.append(x)
            ys.append(y)
        return np.stack(xs), np.stack(ys)
    x1, y1 = get_batch()
    x2, y2 = get_batch()
    return x1, y1, x2, y2

In [508]:
#meta training
opt_init, opt_update = optimizers.adam(step_size=1e-3)
out_shape, net_params = net_init(in_shape)
opt_state = opt_init(net_params)

# vmapped version of maml loss.
# returns scalar for all tasks.
def batch_maml_loss(p, x1_b, y1_b, x2_b, y2_b):
    task_losses = vmap(partial(maml_loss, p))(x1_b, y1_b, x2_b, y2_b)
    return np.mean(task_losses)

@jit
def step(i, opt_state, x1, y1, x2, y2):
    p = optimizers.get_params(opt_state)
    g = grad(batch_maml_loss)(p, x1, y1, x2, y2)
    l = batch_maml_loss(p, x1, y1, x2, y2)
    return opt_update(i, g, opt_state), l

np_batched_maml_loss = []

for i in range(20000):
    x1_b, y1_b, x2_b, y2_b = sample_tasks(num_tasks_sample, batch_size)
    opt_state, l = step(i, opt_state, x1_b, y1_b, x2_b, y2_b)
    np_batched_maml_loss.append(l)
    if i % 1000 == 0:
        print(i)
net_params = optimizers.get_params(opt_state)

ValueError: too many values to unpack (expected 2)

In [509]:
#meta test; train with K examples from validation set on desired task
task='MXL' #the ethnic group we want to test
#pre update prediction
pre_predictions = vmap(partial(net_apply, net_params))(x_test[task])
pre_error= loss(net_params,x_test[task],y_test[task])
#post-update prediction
indx=onp.random.randint(pop[task],batch_size)
x1, y1 = x_train[task][indx] , y_train[task][indx]
for i in range(K):
    opt_state, l = inner_update(net_params,x1,y1)
    
   
post_predictions = vmap(partial(net_apply, net_params))(x_test[task])
post_error= loss(net_params,x_test[task],y_test[task])

print('pre update MSE='+str())
print('post update MSE='+str(error))


NameError: name 'x_test' is not defined

In [ ]:
#heterogeneous model: presence of phenotype if sum allele frequencies is greater than c
#choose thresholds 1,2,3,4,5,6,7,8,9
#choose causal snps (maybe first 5) 
# if the sum of the allele frequencies of these causal snps is greater than the threshold
#then phenotype is 1 otherwise 0



In [21]:
import pandas as pd
import numpy as np

In [2]:
data2=pd.read_csv('processed_data.csv',index_col=[0,1])

In [11]:
data3=data2.apply(lambda x: sum(x),axis=1)

In [63]:
data5=pd.read_csv('eth_ID.csv',index_col=0)

In [70]:
eth=data5['eth'].unique()

In [71]:
eth

array(['GBR', 'FIN', 'EAS', 'PUR', 'CLM', 'IBS', 'PEL', 'SAS', 'KHV',
       'ACB', 'GWD', 'ESN', 'MSL', 'STU', 'EUR', 'YRI', 'JPT', 'LWK',
       'ASW', 'MXL', 'TSI'], dtype=object)

In [99]:
max_threshold=20

In [100]:
threshold_vec= np.random.randint(max_threshold,size=len(eth))

In [101]:
#select 10 random causal snps
heteromodel_causal_snps=np.random.choice(data2.columns,size=10,replace=False)
heteromodel_coef=[]
heteromodel_thresh={}
for i in data2.columns:
    if i in heteromodel_causal_snps:
        heteromodel_coef.append(1)
    else:
        heteromodel_coef.append(0)

j=0
for i in eth:
    heteromodel_thresh[i]=threshold_vec[j]
    j = j + 1

In [142]:
data2.columns

Index(['51479', '55545', '77961', '84002', '87190', '275654', '631490',
       '734210', '779968', '792275',
       ...
       '1325753', '1329159', '1331096', '1334949', '1337898', '1341079',
       '1343267', '1346453', '1351587', '1353091'],
      dtype='object', length=218)

In [144]:
heteromodel_causal_snps=np.random.choice(data2.columns,size=10,replace=False) #choose 10 causal snps from columns of x

In [145]:
heteromodel_causal_snps

array(['1115619', '1160003', '1086657', '1173724', '836587', '981169',
       '1118005', '1337898', '1196039', '1291869'], dtype=object)

In [161]:
data2[]

51479  55545  77961  84002  87190  275654  631490  734210  \
eth ID                                                                   
GBR HG00096      1      0      0      1      0       0       0       0   
    HG00097      0      0      0      0      0       2       0       0   
    HG00099      0      2      0      0      0       0       0       0   
    HG00100      0      0      0      1      0       0       0       0   
    HG00101      0      1      1      0      1       0       0       0   
...            ...    ...    ...    ...    ...     ...     ...     ...   
SAS NA21137      0      0      0      0      0       0       0       0   
    NA21141      1      0      0      1      0       0       0       0   
    NA21142      0      0      0      0      0       1       0       0   
    NA21143      0      0      0      1      0       1       0       0   
    NA21144      0      2      1      0      2       1       0       0   

             779968  792275  ...  1325753  1329159  1331096  1334949  1337898  \
eth ID                       ...                                                
GBR HG00096       2       0  ...        1        2        2        2        1   
    HG00097       2       0  ...        0        2        2        2        2   
    HG00099       2       0  ...        0        2        2        2        2   
    HG00100       2       0  ...        0        2        2        2        2   
    HG00101       2       0  ...        0        2        2        2        2   
...             ...     ...  ...      ...      ...      ...      ...      ...   
SAS NA21137       2       0  ...        0        2        2        2        2   
    NA21141       2       0  ...        1        2        2        2        1   
    NA21142       2       0  ...        0        2        2        2        2   
    NA21143       2       0  ...        0        2        2        2        2   
    NA21144       2       0  ...        1        2        2        2        2   

             1341079  1343267  1346453  1351587  1353091  
eth ID                                                    
GBR HG00096        2        0        0        2        2  
    HG00097        2        0        0        2        2  
    HG00099        2        0        0        2        2  
    HG00100        2        0        0        2        2  
    HG00101        2        0        0        2        2  
...              ...      ...      ...      ...      ...  
SAS NA21137        2        0        0        2        2  
    NA21141        2        0        0        2        2  
    NA21142        2        0        0        2        2  
    NA21143        2        0        0        2        2  
    NA21144        2        0        0        2        2  

[2503 rows x 218 columns]

In [160]:
list(heteromodel_causal_snps)

['1115619',
 '1160003',
 '1086657',
 '1173724',
 '836587',
 '981169',
 '1118005',
 '1337898',
 '1196039',
 '1291869']

In [104]:
#phenotype
data3=data2.apply(lambda x: 1 if x@heteromodel_coef>heteromodel_thresh[x.name[0]] else 0,axis=1)

In [136]:
import numpy as onp

In [133]:
def hetero_model(eth,x,max_threshold=20,num_heteromodel_causal_snps=10):
    threshold_vec= np.random.randint(max_threshold,size=len(eth)) #random choice of thresholds as many as ethnicities
    heteromodel_causal_snps=np.random.choice(x.columns,size=num_heteromodel_causal_snps,replace=False) #choose 10 causal snps from columns of x
    heteromodel_coef=[] #array of 0 or 1 depending on whether a particular column is causal snp or not
    heteromodel_thresh={} #dictionary of thresholds for each ethnicity
    for i in x.columns:
        if i in heteromodel_causal_snps:
            heteromodel_coef.append(1)
        else:
            heteromodel_coef.append(0)

    j=0
    for i in eth:
        heteromodel_thresh[i]=threshold_vec[j]
        j = j + 1
    
    y=x.apply(lambda t: 1 if t@heteromodel_coef>heteromodel_thresh[t.name[0]] else 0,axis=1)
    
    return y 

In [138]:
def linear_model(eth,x):
    #dictionary which contains randomly generated coeffiicnets for the linear model
    eth_coef={}
    eth_errors={}
    eth_errors_sigma2=onp.random.randint(1,5, size=len(eth))
    eth_errors.update([(ethnicity,errors/100) for ethnicity,errors in zip(eth,eth_errors_sigma2)])

    for i in eth:
        eth_coef[i]=onp.random.uniform(-1,1,(x.shape[1]))

    y=x.apply(lambda a: a@eth_coef[a.name[0]]+onp.random.normal(scale=eth_errors[a.name[0]]),axis=1)
    
    return y

In [250]:
def compensatory_model(eth,x,max_threshold1=10,max_threshold2=5,num_heteromodel_causal_snps1=20,num_heteromodel_causal_snps2=50):
    threshold_vec1= np.random.randint(max_threshold1,size=len(eth)) #random choice of thresholds as many as ethnicities
    threshold_vec2= np.random.randint(max_threshold2,size=len(eth)) #random choice of thresholds as many as ethnicities
   
    heteromodel_causal_snps=np.random.choice(x.columns,size=num_heteromodel_causal_snps1+num_heteromodel_causal_snps2,replace=False) #choose 10 causal snps from columns of x
    heteromodel_causal_snps1=heteromodel_causal_snps[0:num_heteromodel_causal_snps1]
    heteromodel_causal_snps2=heteromodel_causal_snps[num_heteromodel_causal_snps1:-1]
    heteromodel_coef1=[] #array of 0 or 1 depending on whether a particular column is causal snp or not
    heteromodel_thresh1={} #dictionary of thresholds for each ethnicity
    for i in x.columns:
        if i in heteromodel_causal_snps1:
            heteromodel_coef1.append(1)
        else:
            heteromodel_coef1.append(0)

    j=0
    for i in eth:
        heteromodel_thresh1[i]=threshold_vec1[j]
        j = j + 1
    
    heteromodel_coef2=[] #array of 0 or 1 depending on whether a particular column is causal snp or not
    heteromodel_thresh2={} #dictionary of thresholds for each ethnicity
    for i in x.columns:
        if i in heteromodel_causal_snps2:
            heteromodel_coef2.append(1)
        else:
            heteromodel_coef2.append(0)

    j=0
    for i in eth:
        heteromodel_thresh2[i]=threshold_vec2[j]
        j = j + 1
    
    y=x.apply(lambda t: 0 if (((t@heteromodel_coef1>heteromodel_thresh1[t.name[0]]) and
              (t@heteromodel_coef2>heteromodel_thresh2[t.name[0]])) | 
                              ((t@heteromodel_coef1<=heteromodel_thresh1[t.name[0]]) and
              (t@heteromodel_coef2<=heteromodel_thresh2[t.name[0]])))
              else 1 ,axis=1)
    
    #return heteromodel_thresh1,heteromodel_thresh2,y
    return y

In [253]:
q1=compensatory_model(eth,data2)

In [255]:
q1[q1==0]

eth  ID     
GBR  HG00096    0
     HG00097    0
     HG00099    0
     HG00100    0
     HG00101    0
               ..
SAS  NA21137    0
     NA21141    0
     NA21142    0
     NA21143    0
     NA21144    0
Length: 2484, dtype: int64

In [226]:
q3=hetero_model(eth,data2)

In [228]:
q3[q3==1]

eth  ID     
FIN  HG00171    1
     HG00173    1
     HG00174    1
     HG00176    1
     HG00177    1
               ..
TSI  NA20822    1
     NA20826    1
     NA20827    1
     NA20828    1
     NA20832    1
Length: 653, dtype: int64

In [175]:
    threshold_vec1= np.random.randint(max_threshold1,size=len(eth)) #random choice of thresholds as many as ethnicities
    threshold_vec2= np.random.randint(max_threshold2,size=len(eth)) #random choice of thresholds as many as ethnicities
   
    heteromodel_causal_snps=np.random.choice(x.columns,size=num_heteromodel_causal_snps1+num_heteromodel_causal_snps2,replace=False) #choose 10 causal snps from columns of x
    heteromodel_causal_snps1=heteromodel_causal_snps[0:num_heteromodel_causal_snps1]
    heteromodel_causal_snps2=heteromodel_causal_snps[num_heteromodel_causal_snps1:-1]
    heteromodel_coef1=[] #array of 0 or 1 depending on whether a particular column is causal snp or not
    heteromodel_thresh1={} #dictionary of thresholds for each ethnicity
    for i in x.columns:
        if i in heteromodel_causal_snps1:
            heteromodel_coef1.append(1)
        else:
            heteromodel_coef1.append(0)

    j=0
    for i in eth:
        heteromodel_thresh1[i]=threshold_vec1[j]
        j = j + 1
    
    heteromodel_coef2=[] #array of 0 or 1 depending on whether a particular column is causal snp or not
    heteromodel_thresh2={} #dictionary of thresholds for each ethnicity
    for i in x.columns:
        if i in heteromodel_causal_snps2:
            heteromodel_coef2.append(1)
        else:
            heteromodel_coef2.append(0)

    j=0
    for i in eth:
        heteromodel_thresh2[i]=threshold_vec2[j]
        j = j + 1
    
    y=x.apply(lambda t: 0 if (t@heteromodel_coef1>heteromodel_thresh1[t.name[0]] and
              t@heteromodel_coef2>heteromodel_thresh2[t.name[0]] | 
                              t@heteromodel_coef1<=heteromodel_thresh1[t.name[0]] and
              t@heteromodel_coef2<=heteromodel_thresh2[t.name[0]])
              else 1,axis=1)

NameError: name 'max_threshold1' is not defined